In [1]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset, random_split
from transformers import AutoImageProcessor, ResNetForImageClassification, TrainingArguments, Trainer

warnings.filterwarnings('ignore')

In [2]:
# Paths
data_path = 'C:/Users/nithi/Desktop/FAU/Semester-4/Master_Thesis_Federated_Learning/Dataset/sample'
csv_path = os.path.join(data_path, 'chest_image_metadata.csv')

In [3]:
# Load metadata
metadata = pd.read_csv(csv_path)

metadata.head()

,Image-ID,Label
0,00000030_001_aug_2_0.png,Atelectasis
1,00000030_001_aug_3_0.png,Atelectasis
2,00000175_001.png,Atelectasis
3,00000175_001_aug_1_0.png,Atelectasis
4,00000175_002.png,Atelectasis


In [4]:
# Custom Dataset class
class PathologyDataset(Dataset):
    def __init__(self, csv_file, img_dir, processor, transform=None):
        self.metadata = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.metadata.iloc[idx, 1], self.metadata.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = classes.index(self.metadata.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.squeeze() for k, v in inputs.items()}
        return {'pixel_values': inputs['pixel_values'], 'labels': torch.tensor(label)}

In [5]:
# Define class names
classes = ['Hernia', 'Pneumonia', 'Fibrosis', 'Nodule', 'Mass', 'Consolidation', 'Effusion', 'Edema', 'Atelectasis',
           'No Finding', 'Cardiomegaly', 'Pneumothorax', 'Pleural_Thickening', 'Infiltration', 'Emphysema']

In [6]:
# Initialize processor
processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50", force_download=False)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [7]:
# Create dataset
dataset = PathologyDataset(csv_file=csv_path, img_dir=data_path, processor=processor)

In [8]:
# Split dataset into train, validation, and test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [9]:
# Load and modify the model
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50", num_labels=len(classes),
                                                     ignore_mismatched_sizes=True)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([15, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.001,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [11]:
# Define compute metrics function
def compute_metrics(p):
    pred_labels = np.argmax(p.predictions, axis=1)
    accuracy = accuracy_score(p.label_ids, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, pred_labels, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [12]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [14]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
test_results = trainer.evaluate(test_dataset)
print(test_results)

In [ ]:
# Predict and evaluate on the test set
predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(-1)
true_labels = [example['Label'].item() for example in test_dataset]

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(true_labels, pred_labels)
ConfusionMatrixDisplay(conf_matrix, display_labels=classes).plot(xticks_rotation='vertical')
plt.show()

In [ ]:
# Function to predict and display an image
def predict_and_display(img_path, true_label):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_label_idx = logits.argmax(-1).item()
    predicted_label = classes[predicted_label_idx]

    plt.imshow(image)
    plt.title(f"True Label: {true_label}\nPredicted Label: {predicted_label}")
    plt.axis("off")
    plt.show()

In [ ]:
# Example usage
example_img_path = os.path.join(data_path, metadata.iloc[0, 1], metadata.iloc[0, 0])
true_label = metadata.iloc[0, 1]
predict_and_display(example_img_path, true_label)